In [1]:
import pickle
import numpy as np

In [2]:
files = ["data/cifar10/train/data_batch_1.bin","data/cifar10/train/data_batch_2.bin","data/cifar10/train/data_batch_3.bin","data/cifar10/train/data_batch_4.bin","data/cifar10/train/data_batch_5.bin"]
# files = ["cifar-10-batches-py/data_batch_1.bin","cifar-10-batches-py/data_batch_2.bin","cifar-10-batches-py/data_batch_3.bin","cifar-10-batches-py/data_batch_4.bin","cifar-10-batches-py/data_batch_5.bin"]

In [3]:
x_train = np.zeros((50000, 3072))
y_train = np.zeros((50000))

In [1]:
import numpy as np
import pickle
import os

# 确保路径正确
files = [
    "data/cifar10/train/data_batch_1.bin",
    "data/cifar10/train/data_batch_2.bin",
    "data/cifar10/train/data_batch_3.bin",
    "data/cifar10/train/data_batch_4.bin",
    "data/cifar10/train/data_batch_5.bin"
]

# 初始化数组
x_train = np.zeros((50000, 3072), dtype=np.uint8)
y_train = np.zeros((50000,), dtype=np.int32)

# 加载数据
i = 0
for file in files:
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
        start = i * 10000
        end = start + 10000
        x_train[start:end, :] = dict[b'data']
        y_train[start:end] = dict[b'labels']
        print(start, end)
        i += 1

print("Data loaded successfully.")


UnpicklingError: invalid load key, '\x06'.

In [5]:
x_test = np.zeros((10000,3072))
y_test = np.zeros((10000))

In [6]:
x_train.shape

(50000, 3072)

In [7]:

file = "data/cifar10/test/test_batch.bin"
with open(file, 'rb') as fo:
    dict = pickle.load(fo, encoding='bytes')
 
    x_test[0:10000,:] = dict[b'data']
    y_test[0:10000] = dict[b'labels']
    
    fo.close()

UnpicklingError: invalid load key, '\x03'.

In [8]:
def unpickle(fileName):
    '''
    Description: retrieve data from CIFAR-10 Pickles
    Params: fileName = filename to unpickle
    Outputs: Unpickled Dict
    '''
    with open(fileName, 'rb') as f:
        dict = pickle.load(f, encoding='bytes')
    return dict


def merge_batches():
    
    for i in range(0,5):
        fileName = "../input/cifar10/data_batch_" + str(i + 1)
        data = unpickle(fileName)
        if i == 0:
            features = data[b'data']
            labels = np.array(data[b'labels'])
        else:
            features = np.append(features, data[b'data'], axis=0)
            labels = np.append(labels, data[b'labels'], axis=0)
    return features, labels


def one_hot_encode(data):
    '''
    Description: Encode Target Label IDs to one hot vector of size L where L is the
    number of unique labels
    Params: data = list of label IDs
    Outputs: List of One Hot Vectors
    '''
    one_hot = np.zeros((data.shape[0], 10))
    one_hot[np.arange(data.shape[0]), data] = 1
    return one_hot


def normalize(data):
    '''
    Description: Normalize Pixel values
    Params: list of Image Pixel Features
    Outputs: Normalized Image Pixel Features
    '''
    return data / 255.0


def preprocess():
    '''
    Description: helper function to load and preprocess CIFAR-10 training data batches
    Params: num_to_load = number of batches of CIFAR-10 to load and merge
    Outputs: Pre-processed CIFAR-10 image features and labels
    '''
    X, y = merge_batches()
    X = normalize(X)
    X = X.reshape(-1, 3072, 1)
    y = one_hot_encode(y)
    y = y.reshape(-1, 10, 1)
    return X, y


def dataset_split(X, y, ratio=0.9):
    '''
    Description: helper function to split training data into training and validation
    Params: X=image features
            y=labels
            ratio = ratio of training data from total data
    Outputs: training data (features and labels) and validation data
    '''
    split = int(ratio * X.shape[0])
    indices = np.random.permutation(X.shape[0])
    training_idx, val_idx = indices[:split], indices[split:]
    X_train, X_val = X[training_idx, :], X[val_idx, :]
    y_train, y_val = y[training_idx, :], y[val_idx, :]
    print("Records in Training Dataset", X_train.shape[0])
    print("Records in Validation Dataset", X_val.shape[0])
    return X_train, y_train, X_val, y_val


def sigmoid(out):
    '''
    Description: Sigmoid Activation
    Params: out = a list/matrix to perform the activation on
    Outputs: Sigmoid activated list/matrix
    '''
    return 1.0 / (1.0 + np.exp(-out))


def delta_sigmoid(out):
    '''
    Description: Derivative of Sigmoid Activation
    Params: out = a list/matrix to perform the activation on
    Outputs: Delta(Sigmoid) activated list/matrix
    '''
    return sigmoid(out) * (1 - sigmoid(out))

def SigmoidCrossEntropyLoss(a, y):

        return np.sum(np.nan_to_num(-y*np.log(a)-(1-y)*np.log(1-a)))

In [9]:
class DNN(object):
    '''
            Description: Class to define the Deep Neural Network
    '''

    def __init__(self, sizes):
        '''
        Description: initialize the biases and weights using a Gaussian
        distribution with mean 0, and variance 1.
        Biases are not set for 1st layer that is the input layer.
        Params: sizes = a list of size L; where L is the number of layers
                        in the deep neural network and each element of list contains
                        the number of neuron in that layer.
                        first and last elements of the list corresponds to the input
                        layer and output layer respectively
                        intermediate layers are hidden layers.
        '''
        self.num_layers = len(sizes)
        # setting appropriate dimensions for weights and biases
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def forwardPropagation(self, x):
        '''
        Description: Forward Passes an image feature matrix through the Deep Neural
                                 Network Architecture.
        Params: x = Image Features
        Outputs: 2 lists which stores outputs and activations at every layer,
                 1st list is non-activated and 2nd list is activated
                 The last element of the 2nd list corresponds to the scores against
                 10 labels in the dataset.
        '''
        activation = x
        activations = [x]  # list to store activations for every layer
        outs = []  # list to store out vectors for every layer
        for b, w in zip(self.biases, self.weights):
            out = np.dot(w, activation) + b
            outs.append(out)
            activation = sigmoid(out)
            activations.append(activation)
        return outs, activations

    def get_batch(self, X, y, batch_size):
        '''
        Description: A data iterator for batching of image features and labels
        Params: X, y = lists of Features and corresponding labels, these lists
                                   have to be batched.
                batch_size =  size of the batch
        Outputs: a batch of image features and labels of size = batch_size
        '''
        for batch_idx in range(0, X.shape[0], batch_size):
            batch = zip(X[batch_idx:batch_idx + batch_size],
                        y[batch_idx:batch_idx + batch_size])
            yield batch

    def train(self, X, y,x_val,y_val ,batch_size=100, learning_rate=0.01, epochs=100):
        '''
        Description: Batch-wise trains image features against corresponding labels.
                     The weights and biases of the neural network are updated through
                     backpropagation on batches using SGD
                     del_b and del_w are of same size as all the weights and biases
                     of all the layers. del_b and del_w contains the gradients which
                     are used to update weights and biases
        Params: X, y = lists of training features and corresponding labels
                batch_size =  size of the batch
                learning_rate = eta; controls the size of changes in weights & biases
                epochs = no. of times to iterate of the whole data
        '''
        n_batches = X.shape[0] // batch_size
        for j in range(epochs):
            batch_iter = self.get_batch(X, y, batch_size)
            #print(type(batch_iter))
            for i in range(n_batches):
                batch = next(batch_iter)
                # same shape as self.biases
                del_b = [np.zeros(b.shape) for b in self.biases]
                # same shape as self.weights
                del_w = [np.zeros(w.shape) for w in self.weights]
                for batch_X, batch_y in batch:
                    # accumulate all the bias and weight gradients
                    loss, delta_del_b, delta_del_w = self.backPropagation(
                        batch_X, batch_y)
                    del_b = [db + ddb for db, ddb in zip(del_b, delta_del_b)]
                    del_w = [dw + ddw for dw, ddw in zip(del_w, delta_del_w)]
            # update weight and biases by multiplying ratio learning rate and batch_size
            # multiplied with the accumulated gradients(partial derivatives)
            # calculate change in weight(delta) and biases and update weight
            # with the changes
            self.weights = [w - (learning_rate / batch_size)
                            * delw for w, delw in zip(self.weights, del_w)]
            self.biases = [b - (learning_rate / batch_size)
                           * delb for b, delb in zip(self.biases, del_b)]
            acc_val = self.eval(x_val,y_val)
            acc_train = self.eval(X,y)
            
            print("\nEpoch %d complete\t training accuracy: %f\n"%(j, acc_train))
            print("\nEpoch %d complete\t validation accuracy: %f\n"%(j, acc_val))
            print("\nEpoch %d complete\t taining Loss: %f\n"%(j, loss))

    def backPropagation(self, x, y):

        del_b = [np.zeros(b.shape) for b in self.biases]
        del_w = [np.zeros(w.shape) for w in self.weights]


        outs, activations = self.forwardPropagation(x)

        loss = SigmoidCrossEntropyLoss(activations[-1],y)

        delta_cost = activations[-1] - y

        delta = delta_cost
        del_b[-1] = delta
        del_w[-1] = np.dot(delta, activations[-2].T)


        for l in range(2, self.num_layers):
            out = outs[-l]
            delta_activation = delta_sigmoid(out)
            
            delta = np.dot(self.weights[-l + 1].T, delta) * delta_activation
            del_b[-l] = delta
            del_w[-l] = np.dot(delta, activations[-l - 1].T)
        return (loss, del_b, del_w)

    def eval(self, X, y):
        '''
        Description: Based on trained(updated) weights and biases, predict a label and compare
                     it with original label and calculate accuracy
        Params: X, y = a data example from validation dataset (image features, labels)
        Outputs: accuracy of prediction
        '''
        count = 0
        for x, _y in zip(X, y):
            outs, activations = self.forwardPropagation(x)
            # postion of maximum value is the predicted label
            if np.argmax(activations[-1]) == np.argmax(_y):
                count += 1
        
        
        #print("Accuracy: %f" % ((float(count) / X.shape[0]) * 100))
        return float(count) / X.shape[0] * 100

    def predict(self, X):
        '''
        Description: Based on trained(updated) weights and biases, predict a label for an
                                 image which does not have a label.
        Params: X = list of features of unknown images
        Outputs: list containing the predicted label for the corresponding unknown image
        '''
        labels = unpickle("cifar-10-batches-py/batches.meta")["label_names"]
        preds = np.array([])
        for x in X:
            outs, activations = self.feedforward(x)
            preds = np.append(preds, np.argmax(activations[-1]))
        preds = np.array([labels[int(p)] for p in preds])
        return preds

In [10]:
X, y = preprocess()
X_train, y_train, X_val, y_val = dataset_split(X, y)
    
model = DNN([3072,30, 20, 10])  # initialize the model
model.train(X_train, y_train,X_val, y_val, epochs=5)  # train the model


FileNotFoundError: [Errno 2] No such file or directory: '../input/cifar10/data_batch_1'